In [16]:
import torch
from lightning import LightningModule, Trainer
from dataloader import InsectDatamodule

In [32]:
import torch
from lightning import LightningModule

class ResBlock(torch.nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, n_max_pool: int, **kwargs):
        super().__init__()
        padding = kernel_size // 2

        self.conv1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding='same', **kwargs)
        self.batchnorm1 = torch.nn.BatchNorm2d(num_features=out_channels)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding='same', **kwargs)
        self.batchnorm2 = torch.nn.BatchNorm2d(num_features=out_channels)
        self.maxpool = torch.nn.MaxPool2d(kernel_size=n_max_pool, stride=n_max_pool)

    def forward(self, x):
        out = self.conv1(x)
        out = self.batchnorm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.batchnorm2(out)
        out = self.relu(out) + x
        return self.maxpool(out)


class ResNet(LightningModule):
    def __init__(
            self,
            in_channels: int,
            out_channels: int,
            kernel_size: int,
            n_max_pool: int,
            n_res_blocks: int,
            num_classes: int,
            learning_rate: float = 0.001,
            **kwargs):
        super().__init__()

        self.learning_rate = learning_rate

        self.conv1 = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding='same', **kwargs)
        self.batchnorm1 = torch.nn.BatchNorm2d(num_features=out_channels)
        self.relu = torch.nn.ReLU()
        self.res_blocks = torch.nn.Sequential(
            *[ResBlock(
                in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, n_max_pool=n_max_pool, **kwargs) for _ in range(n_res_blocks)]
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.convout = torch.nn.Conv2d(in_channels=out_channels, out_channels=num_classes, kernel_size=1, **kwargs)

        self.softmax = torch.nn.Softmax(dim=1)

        self.cross_entropy_loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        # x: (N, C=1, H, W)
        x = x.unsqueeze(1)

        # Run input through a first convolutional layer.
        out = self.conv1(x)
        out = self.batchnorm1(out)
        out = self.relu(out)

        # Run input through the residual blocks.
        out = self.res_blocks(out)

        # Run input through the average pooling layer. The output is a tensor of shape (N, C, 1, 1).
        out = self.avgpool(out)

        # Run input through the output convolutional layer. This is the same as a fully connected layer but it works with 4D tensors.
        out = self.convout(out)

        # Flatten the output tensor to have shape (N, C).
        out  = out.flatten(1)

        out = self.softmax(out)

        return out

    def accuracy(self, y_hat, y):
        labels_hat = torch.argmax(y_hat, dim=1)
        labels = torch.argmax(y, dim=1)
        return torch.sum(labels == labels_hat).item() / (len(y) * 1.0)

    def cross_entropy(self, y_hat, y):
        return self.cross_entropy_loss_fn(y_hat, y.float())

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch

        y_hat = self(x)

        loss = self.cross_entropy(y_hat, y)

        # Logging to TensorBoard (if installed) by default
        self.log_dict({'train_loss': loss, 'train_acc': self.accuracy(y_hat, y)})

        return loss

    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch

        y_hat = self(x)

        loss = self.cross_entropy(y_hat, y)

        # Logging to TensorBoard (if installed) by default
        self.log_dict({'val_loss': loss, 'val_acc': self.accuracy(y_hat, y)})

        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=0.0)

In [33]:
# initialize the datamodule

csv_paths = ['../data/Cicadidae.csv', '../data/Orthoptera.csv']

batch_size = 10
num_workers = 0

n_fft = 1000
hop_length =  147
win_length = int(hop_length * 2)
n_mels = 64

datamodule = InsectDatamodule(
    csv_paths = csv_paths, batch_size = batch_size, num_workers = num_workers, hop_length = hop_length, n_fft = n_fft, win_length = win_length, n_mels = n_mels)

resnet = ResNet(
    in_channels=1, out_channels=32, kernel_size=3, n_max_pool=3, n_res_blocks=3, num_classes=datamodule.num_classes)

In [34]:
trainer = Trainer()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [35]:
trainer.fit(resnet, train_dataloaders=datamodule.train_dataloader(), val_dataloaders=datamodule.val_dataloader())

c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\loops\utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type              | Params
------------------------------------------------------------
0 | conv1                 | Conv2d            | 320   
1 | batchnorm1            | BatchNorm2d       | 64    
2 | relu                  | ReLU              | 0     
3 | res_blocks            | Sequential        | 55.9 K
4 | avgpool               | AdaptiveAvgPool2d | 0     
5 | convout               | Conv2d            | 1.1 K 
6 | softmax               | Softmax           | 0     
7 | cross_entropy_loss_fn | CrossEntropyLoss  | 0     
------------------------------------------------------------
57.3 K    Trainable params
0         Non-trainable params
57.3 K    Total params
0.229     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\kraft\.conda\envs\torch_cuda\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 48:  38%|███▊      | 8/21 [00:02<00:03,  3.62it/s, v_num=2] 